In [1]:
import sys
sys.path.append('../')

In [3]:
from scripts.packages.yolo_predict import YOLO_Pred
import cv2
import yaml
from yaml.loader import SafeLoader
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
print(cv2.__version__)

4.6.0


In [ ]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [22]:
df_predictions = yolo.predictions('C:/Users/admin/Downloads/eudes-fede/eudes-fede/test_8.jpg')

image = cv2.imread('C:/Users/admin/Downloads/eudes-fede/eudes-fede/test_8.jpg')
image_w, image_h = image.shape[:2]
print(image_w, image_h )
INPUT_WH_YOLO=640
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO
print(x_factor, y_factor)

df_predictions[['Width']]=df_predictions[['Width']]/x_factor
df_predictions[['Height']]=df_predictions[['Height']]/x_factor

3264 2448
5.1 3.825


#### 1er algoritmo/heuristica


    * Detecta bounding box vecino de la izquierda
        * TODO --> mismos calculos pero para la derecha y arriba


In [48]:
df_predictions.loc[158]

Center_X    204.798553
Center_Y    269.719086
Width        37.879337
Height       46.832764
Name: 158, dtype: float64

In [40]:
df_predictions.loc[598 ]

Center_X    171.735123
Center_Y    266.884888
Width        29.631151
Height       46.749153
Name: 598, dtype: float64

In [60]:
#df_predictions / 
dict_l_n = {  } # neigh left
dict_r_n = { } # neigh right
dict_h_n= {}  # alone right
dict_a = { } # alone right


for index_bb in df_predictions.index:
    list_of_neightbours_l = []
    list_of_neightbours_r = []
    list_of_neightbours_h = [] 
    list_of_alones_l = []
    list_of_alones_r = []
    list_of_alones_h=[]
    for index in df_predictions.index:
        X_center_0 = df_predictions.loc[index_bb][0]
        Y_center_0 = df_predictions.loc[index_bb][1]
        Width_0 = df_predictions.loc[index_bb][2]
        height_0 = df_predictions.loc[index_bb][3]
        threshold_x_0 = 0.8* Width_0/x_factor
        threshold_y_0 = 0.8 * height_0/y_factor

        X_center_neightbour = df_predictions.loc[index][0]
        Y_center_neightbour = df_predictions.loc[index][1]
        a=4
        k=1.2

        ### Neightbor Left
        x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
        x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

        y_min_l = Y_center_0 - (k*threshold_y_0)
        y_max_l = Y_center_0 + (k*threshold_y_0)
        
        ### Neightbor Right
        x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
        x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

        y_min_r = Y_center_0 - (k*threshold_y_0)
        y_max_r = Y_center_0 + (k*threshold_y_0)
        
        ### Neightbor High
        x_min_h = X_center_0 - (k*threshold_x_0)
        x_max_h = X_center_0 + (2*k*threshold_x_0)

        y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
        y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)
        
        ### Search neig left
        if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
            None
            if index_bb==694 and index==693:
                print(f'El bounding box {index_bb} tiene al bb {index} de vecino')
                print(x_min_l, X_center_neightbour, x_max_l, "___", y_min_l, Y_center_neightbour,y_max_l )
            list_of_neightbours_l.append([index])
            try:
                dict_l_n[str(index_bb)].append(index)
            except KeyError:
                dict_l_n[str(index_bb)]=[]
                dict_l_n[str(index_bb)].append(index)
            
        else:
            if index_bb==288 and index==277:
                print(f'El bounding box {index_bb} no tiene vencidad con {index}')
                print(x_min_l, X_center_neightbour, x_max_l, "___", y_min_l, Y_center_neightbour,y_max_l )
            list_of_alones_l.append([index])
                # print(x_min, X_center_neightbour, x_max, "___", y_min, Y_center_neightbour,y_max )
        ### Search neig right
        if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
            None
            # if index_bb==1060:
            #     print(f'El bounding box {index_bb} tiene al bb {index} de vecino')
            #     print(x_min, X_center_neightbour, x_max, "___", y_min, Y_center_neightbour,y_max )
            list_of_neightbours_r.append([index])
            try:
                dict_r_n[str(index_bb)].append(index)
            except KeyError:
                dict_r_n[str(index_bb)]=[]
                dict_r_n[str(index_bb)].append(index)
        else:
            None
            # if index_bb==1060 and index==1059:
            #     print(f'El bounding box {index_bb} no tiene vecinos')
            list_of_alones_r.append([index_bb, index])
            
        # Search neig hight    
        if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
            None
            # if index_bb==1060:
            #     print(f'El bounding box {index_bb} tiene al bb {index} de vecino')
            #     print(x_min, X_center_neightbour, x_max, "___", y_min, Y_center_neightbour,y_max )
            list_of_neightbours_h.append([index])
            try:
                dict_h_n[str(index_bb)].append(index)
            except KeyError:
                dict_h_n[str(index_bb)]=[]
                dict_h_n[str(index_bb)].append(index)
        else:
            None
            # if index_bb==1060 and index==1059:
            #     print(f'El bounding box {index_bb} no tiene vecinos')
            list_of_alones_h.append([index_bb, index])
    # try:
    #     dict_a[str(index_bb)].append(index)
    # except KeyError:
    #     dict_a[str(index_bb)]=indexSSS
    #             # print(x_min, X_center_neightbour, x_max, "___", y_min, Y_center_neightbour,y_max )
    

El bounding box 694 tiene al bb 693 de vecino
251.52227435392493 262.4385681152344 271.8318852368523 ___ 177.3695993460861 192.5100555419922 204.15035915000766


In [69]:
print(len(dict_r_n))
dict_r_n['664']

55


[368]

In [ ]:
dict_l_n

{'303': [606],
 '368': [664],
 '333': [176],
 '288': [277],
 '693': [576],
 '277': [276],
 '694': [693],
 '594': [309],
 '370': [368],
 '341': [494],
 '276': [126],
 '576': [288],
 '606': [141, 319],
 '602': [158],
 '609': [608],
 '158': [598],
 '598': [315, 748],
 '126': [274, 422],
 '349': [336],
 '207': [537],
 '374': [215],
 '245': [77],
 '268': [694],
 '102': [5],
 '608': [303],
 '272': [112],
 '283': [268],
 '5': [4],
 '376': [374],
 '400': [677],
 '494': [493],
 '688': [80],
 '84': [93, 688],
 '261': [96, 97],
 '336': [333],
 '461': [594],
 '360': [514],
 '699': [283],
 '325': [175],
 '630': [36],
 '94': [409],
 '502': [171],
 '274': [272],
 '748': [461],
 '42': [502],
 '551': [684],
 '200': [199],
 '149': [609],
 '96': [103],
 '403': [551],
 '215': [372],
 '4': [2],
 '179': [333, 176],
 '493': [757],
 '409': [84],
 '97': [103],
 '103': [7],
 '319': [602],
 '67': [391]}

ERROR! Session/line number was not unique in database. History logging moved to new session 512


In [31]:
dict_h_n

{'594': [336],
 '276': [748],
 '84': [694],
 '699': [149],
 '757': [372],
 '215': [224],
 '243': [276],
 '80': [576],
 '224': [390],
 '199': [222]}

In [133]:
print(len(list_of_neightbours_l), len(list_of_neightbours_r))

11 17


In [138]:
i=757
print("list_of_alones_l: ", i in list_of_alones_l)
print("list_of_neightbours_l", i in list_of_neightbours_l)
print("list_of_alones_r: ", i in list_of_alones_r)
print("list_of_neightbours_r", i in list_of_neightbours_r)


list_of_alones_l:  True
list_of_neightbours_l False
list_of_alones_r:  False
list_of_neightbours_r False


In [149]:
list_of_neightbours

[333,
 333,
 693,
 693,
 310,
 309,
 36,
 694,
 276,
 576,
 606,
 609,
 598,
 207,
 374,
 268,
 272,
 5,
 5,
 494,
 461,
 360,
 748,
 748,
 757,
 42,
 96,
 67]

In [33]:
## Plot Neighbors

for ind in [158,319, 315, 748]:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    # bb_conf = int(confidences_np[ind]*100)
    # classes_id = classes[ind]
    # class_name = self.labels[classes_id]
    colors = (255, 0, 0)

    # text = f'{class_name}: {bb_conf}%'

    cv2.rectangle(image,(x,y),(x+w,y+h),colors,2)
    cv2.rectangle(image,(x,y-30),(x+w,y),colors,-1)

    cv2.putText(image,str(ind),(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
cv2.imwrite(filename='test_neightbours_r.jpg',img=image)

True

### IoU

In [88]:
a=276
xA1=df_predictions.loc[a][0]-df_predictions.loc[a][2]/2
yA1=df_predictions.loc[a][1]+df_predictions.loc[a][3]/2
xA2=df_predictions.loc[a][0]+df_predictions.loc[a][2]/2
yA2=df_predictions.loc[a][1]-df_predictions.loc[a][3]/2
boxA=[xA1, yA1, xA2, yA2 ]

b=277
xB1=df_predictions.loc[b][0]-df_predictions.loc[b][2]/2
yB1=df_predictions.loc[b][1]+df_predictions.loc[b][3]/2
xB2=df_predictions.loc[b][0]+df_predictions.loc[b][2]/2
yB2=df_predictions.loc[b][1]-df_predictions.loc[b][3]/2
boxB=[xB1, yB1,xB2, yB2 ]


In [80]:
# import the necessary packages
from collections import namedtuple
import numpy as np
import cv2

# define the Detection object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [89]:
bb_intersection_over_union(boxA, boxB)

-0.012425531609167425

In [ ]:
left
X_c_e=X_c-w
Y_c_e=Y_c

Right
X_c_e=X_c+w
Y_c_e=Y_c